In [ ]:
import os

import tqdm
import numpy as np
import tensorflow as tf
import tensorflow_gnn as tfgnn
import tensorflow_ranking as tfr

from tpu_graphs.baselines.layout.data import get_npz_split, get_npz_dataset
from tpu_graphs.baselines.layout.models import ResModel
from gfos.data.utils import load_layout
from gfos.metrics import metric_for_layout_collections


In [ ]:
data_root_dir = r"H:\data\gfos\predict-ai-model-runtime\npz_all\npz\layout\xla\random"
num_configs = 16
max_configs = 1000
batch_size = 8

In [ ]:
# valid_dataset = get_npz_split(
#     os.path.join(data_root_dir, "valid"),
#     min_configs=num_configs,
#     max_configs=max_configs,
#     cache_dir="../../.cache",
# )
data_partition = get_npz_dataset(
    data_root_dir,
    min_train_configs=num_configs,
    max_train_configs=max_configs,
    cache_dir="../../.cache",
)

In [ ]:
def _graph_and_label(graph: tfgnn.GraphTensor):
    # Return runtimes divded over large number: only ranking is required. The
    # runtimes are in the 100K range
    label = tf.cast(graph.node_sets["g"]["runtimes"], tf.float32) / 1e7
    return graph, label

valid_ds = (
    data_partition.validation.get_graph_tensors_dataset(num_configs)
    .batch(batch_size, drop_remainder=False)
    .map(tfgnn.GraphTensor.merge_batch_to_components)
    .map(_graph_and_label)
)

In [ ]:
model = ResModel(16, 119)

loaded = tf.keras.models.load_model(
    "../../src/tpu_graphs/output/model_5260f25ba9d0eae9c5e563a16848fd08",
    custom_objects={"opa_metric": tfr.keras.metrics.OPAMetric},
)

model.forward(subconfigs_graph, num_configs=100, backprop=False);

for v, lv in zip(model.variables, loaded.variables):
    v.assign(lv.value())

In [ ]:
_INFERENCE_CONFIGS_BATCH_SIZE = 100

test_rankings = []

assert data_partition.validation.graph_id is not None

for graph in tqdm.tqdm(
    data_partition.validation.iter_graph_tensors(),
    total=data_partition.validation.graph_id.shape[-1],
    desc="Inference",
):
    num_configs = graph.node_sets["g"]["runtimes"].shape[-1]
    all_scores = []

    for i in range(0, num_configs, _INFERENCE_CONFIGS_BATCH_SIZE):
        end_i = min(i + _INFERENCE_CONFIGS_BATCH_SIZE, num_configs)
        # Take a cut of the configs.
        node_set_g = graph.node_sets["g"]
        subconfigs_graph = tfgnn.GraphTensor.from_pieces(
            edge_sets=graph.edge_sets,
            node_sets={
                "op": graph.node_sets["op"],
                "nconfig": tfgnn.NodeSet.from_fields(
                    sizes=graph.node_sets["nconfig"].sizes,
                    features={
                        "feats": graph.node_sets["nconfig"]["feats"][
                            :, i:end_i
                        ],
                    },
                ),
                "g": tfgnn.NodeSet.from_fields(
                    sizes=tf.constant([1]),
                    features={
                        "graph_id": node_set_g["graph_id"],
                        "runtimes": node_set_g["runtimes"][:, i:end_i],
                        "kept_node_ratio": node_set_g["kept_node_ratio"],
                    },
                ),
            },
        )
        h = model.forward(
            subconfigs_graph, num_configs=(end_i - i), backprop=False
        )
        all_scores.append(h[0])

    all_scores = tf.concat(all_scores, axis=0)
    graph_id = graph.node_sets["g"]["graph_id"][0].numpy().decode()
    sorted_indices = (
        tf.strings.join(tf.strings.as_string(tf.argsort(all_scores)), ";")
        .numpy()
        .decode()
    )
    test_rankings.append((graph_id, sorted_indices))

In [ ]:
LAYOUT_DIR = r"H:\data\gfos\predict-ai-model-runtime\npz_all\npz\layout"
layouts = load_layout(LAYOUT_DIR, model_type="xla", compile_type="random")


In [ ]:

pred_dict = dict(test_rankings)
scores = []
for file in layouts["valid"]:
    filename = os.path.basename(file)[:-4]
    idx = pred_dict[filename].split(";")
    idx = [int(i) for i in idx]
    
    runtime = np.load(file)["config_runtime"]
    pred = runtime[idx]
    gt = runtime[np.argsort(runtime)]
    
    score = metric_for_layout_collections(pred, gt[:len(pred)])
    scores.append(score)


In [ ]:
np.mean(scores)